In [129]:
import pandas as pd
import numpy as np
from ast import literal_eval
import jieba
pd.set_option('display.max_columns',None)
import os
import jieba.posseg as pseg
from collections import Counter
import re
import warnings
warnings.filterwarnings('ignore')

### 1. Data Loading

In [130]:
all_docs = pd.read_csv('all_docs.txt',sep= '\001',header = None)

In [131]:
all_docs.columns = ['ID','title','content']

In [132]:
all_docs.head()

,ID,title,content
0,D000001,林志颖老婆深夜敷面膜，睫毛太长好吓人,早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...
1,D000002,小s夸杨幂身材好，杨幂回复太精彩了！,翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...
2,D000003,年轻时的她风华绝代，现却无人送祝福,上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...
3,D000004,林心如屡曝霍建华私生活被怼蹭老公人气,霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...
4,D000005,曾是TVB颜值担当，近照曝光发现真老了,不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...


In [133]:
train_docs_keywords = pd.read_csv('train_docs_keywords.txt',sep= '\t',header = None)

In [134]:
train_docs_keywords.columns = ['ID','Tag']

In [135]:
train_docs_keywords.head()

,ID,Tag
0,D012650,"三生三世,凤九,东华"
1,D047118,"南宁,美食"
2,D098970,"凉菜,北京人"
3,D092010,"华为,P30pro"
4,D103408,"酒吧,世界杯"


In [136]:
docs_tag = pd.merge(all_docs,train_docs_keywords,how = 'left',on = 'ID')

In [137]:
pred_data = docs_tag[docs_tag['Tag'].isnull()]

In [138]:
pred_data.head()

,ID,title,content,Tag
0,D000001,林志颖老婆深夜敷面膜，睫毛太长好吓人,早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...,NaN
1,D000002,小s夸杨幂身材好，杨幂回复太精彩了！,翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...,NaN
2,D000003,年轻时的她风华绝代，现却无人送祝福,上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...,NaN
3,D000004,林心如屡曝霍建华私生活被怼蹭老公人气,霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...,NaN
4,D000005,曾是TVB颜值担当，近照曝光发现真老了,不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...,NaN


In [139]:
train_data = docs_tag[~docs_tag['Tag'].isnull()]

In [140]:
train_data.head(10)

,ID,title,content,Tag
147,D000148,水木年华被嘲讽已过气卢庚戌回应：还好我们还没有断气！,水木年华是内地曾经的第一组合，他们都是出自清华大学。清华大学是很多莘莘学子梦寐以求的最高殿堂...,"水木年华,卢庚戌,过气"
209,D000210,著名导演陈烈院线电影《爱是永恒》即将上映,《甄嬛传》想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先...,"爱是永恒,陈列"
211,D000212,复仇者联盟3：无限战争结局，如何影响漫威影集神盾局特工,《甄嬛传》想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先...,"复仇者联盟3,无限战争,漫威,神盾局特工"
339,D000340,《爱情的边疆》看片受好评殷桃回归质朴,纯爱史诗《爱情的边疆》5月8日将登陆浙江卫视黄金档。五一小长假结束后的第一天，出品方在中国传...,"爱情的边疆,殷桃"
345,D000346,《经常请吃饭的漂亮姐姐》1分钟吸睛桥段诞生丁海寅孙艺珍“对峙”吉海妍,"人民网讯JTBC热播韩剧《经常请吃饭的漂亮姐姐》第10集公开了最为吸睛的1分钟桥段,这段来自...","经常请吃饭的漂亮姐姐,丁海寅,孙艺珍,吉海妍,桥段"
406,D000407,赵丽颖《知否知否》和杨幂《扶摇》假如撞档，你会不会同时追？,最近有人问：赵丽颖《知否知否》和杨幂《扶摇》假如撞档，你会不会同时追？其实《知否》和《扶摇》...,"赵丽颖,知否知否,杨幂,扶摇"
430,D000431,他，绝对是我们可以爱很久的演员,第十放映室就爱瞎说大实话五一档，爱情片《后来的我们》，票房遥遥领先。无论你是冲着导演刘若英，...,"小晓,林见清,后来的我们"
523,D000524,“不老女神”秦海璐素颜扮可爱：今天负责萌萌哒,秦海璐素颜自拍新浪娱乐讯5月3日，秦海璐[微博]在自己微博上传自拍，配文称：“今天不工作，负...,"秦海璐,自拍,女神"
530,D000531,靠着退票成就票房？刘若英靠情怀人设能走向成功？,晚上好，姨太的小可爱们！万万没想到，前两天还成为话题佼佼者的《后来的我们》，在昨天却卷入了轰...,"刘若英,票房,退票"
579,D000580,《真爱的谎言之破冰者》靳远套路乔梁，蔡炳坤套路逗逗,【最走心的影视人物分析，最独特的影评剧评交流】《真爱的谎言之破冰者》里的角色都开始调皮起来。...,"乔梁,真爱的谎言之破冰者,"


### 2. Data Preparation

#### 1) 获取书名号内容

In [141]:
def extract_title(df):
    pattern = '《.*?》'
    title = re.findall(pattern,df,re.S)
    title = np.unique(title)
    return title.tolist()

In [142]:
extract_title('韩红蔡依林合作《舞娘》跨文化碰撞而成《舞娘》极致幻境')

['《舞娘》']

In [143]:
bookName_list = train_data['title'].apply(lambda x:extract_title(str(x)))

In [144]:
bookName_list_1 = []
for book in bookName_list:
    try:
        book_1 = book[0][1:-1]
    except IndexError:
        book_1 = ''
    bookName_list_1.append(book_1)

In [145]:
train_data['BookName'] = bookName_list_1

In [146]:
train_data.head()

,ID,title,content,Tag,BookName
147,D000148,水木年华被嘲讽已过气卢庚戌回应：还好我们还没有断气！,水木年华是内地曾经的第一组合，他们都是出自清华大学。清华大学是很多莘莘学子梦寐以求的最高殿堂...,"水木年华,卢庚戌,过气",
209,D000210,著名导演陈烈院线电影《爱是永恒》即将上映,《甄嬛传》想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先...,"爱是永恒,陈列",爱是永恒
211,D000212,复仇者联盟3：无限战争结局，如何影响漫威影集神盾局特工,《甄嬛传》想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先...,"复仇者联盟3,无限战争,漫威,神盾局特工",
339,D000340,《爱情的边疆》看片受好评殷桃回归质朴,纯爱史诗《爱情的边疆》5月8日将登陆浙江卫视黄金档。五一小长假结束后的第一天，出品方在中国传...,"爱情的边疆,殷桃",爱情的边疆
345,D000346,《经常请吃饭的漂亮姐姐》1分钟吸睛桥段诞生丁海寅孙艺珍“对峙”吉海妍,"人民网讯JTBC热播韩剧《经常请吃饭的漂亮姐姐》第10集公开了最为吸睛的1分钟桥段,这段来自...","经常请吃饭的漂亮姐姐,丁海寅,孙艺珍,吉海妍,桥段",经常请吃饭的漂亮姐姐


### 1) Remove book Name from title and content

In [147]:
def removeBookName(string):
    new_sting = re.sub(r'《.*?》','',string)
    return new_sting

In [148]:
new_title = train_data.apply(lambda x:removeBookName(x[1]),axis = 1)

In [149]:
new_content = train_data.apply(lambda x:removeBookName(x[2]),axis = 1)

In [150]:
train_data['title'] = new_title
train_data['content'] = new_content

In [151]:
train_data.head()

,ID,title,content,Tag,BookName
147,D000148,水木年华被嘲讽已过气卢庚戌回应：还好我们还没有断气！,水木年华是内地曾经的第一组合，他们都是出自清华大学。清华大学是很多莘莘学子梦寐以求的最高殿堂...,"水木年华,卢庚戌,过气",
209,D000210,著名导演陈烈院线电影即将上映,想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先来看看甄嬛...,"爱是永恒,陈列",爱是永恒
211,D000212,复仇者联盟3：无限战争结局，如何影响漫威影集神盾局特工,想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先来看看甄嬛...,"复仇者联盟3,无限战争,漫威,神盾局特工",
339,D000340,看片受好评殷桃回归质朴,纯爱史诗5月8日将登陆浙江卫视黄金档。五一小长假结束后的第一天，出品方在中国传媒大学小礼堂举...,"爱情的边疆,殷桃",爱情的边疆
345,D000346,1分钟吸睛桥段诞生丁海寅孙艺珍“对峙”吉海妍,"人民网讯JTBC热播韩剧第10集公开了最为吸睛的1分钟桥段,这段来自于尹珍雅(孙艺珍饰)和徐...","经常请吃饭的漂亮姐姐,丁海寅,孙艺珍,吉海妍,桥段",经常请吃饭的漂亮姐姐


### 1) Stop Word 过滤

In [152]:
stopwords=pd.read_csv("stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

In [153]:
sentences = []
def preprocess_text(content):
        segs=jieba.lcut(content)
        segs = filter(lambda x:len(x)>1, segs)
        segs = filter(lambda x:x not in stopwords, segs)
        return str((list(segs)))

In [154]:
content_without_stopwords = train_data['content'].apply(preprocess_text)

In [155]:
train_data['content_wordList'] = content_without_stopwords

In [156]:
train_data.head()

,ID,title,content,Tag,BookName,content_wordList
147,D000148,水木年华被嘲讽已过气卢庚戌回应：还好我们还没有断气！,水木年华是内地曾经的第一组合，他们都是出自清华大学。清华大学是很多莘莘学子梦寐以求的最高殿堂...,"水木年华,卢庚戌,过气",,"['水木年华', '内地', '第一', '组合', '出自', '清华大学', '清华大学..."
209,D000210,著名导演陈烈院线电影即将上映,想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先来看看甄嬛...,"爱是永恒,陈列",爱是永恒,"['想必', '二刷', '三刷', '剧中', '位小主', '命运', '差异', '好..."
211,D000212,复仇者联盟3：无限战争结局，如何影响漫威影集神盾局特工,想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先来看看甄嬛...,"复仇者联盟3,无限战争,漫威,神盾局特工",,"['想必', '二刷', '三刷', '剧中', '位小主', '命运', '差异', '好..."
339,D000340,看片受好评殷桃回归质朴,纯爱史诗5月8日将登陆浙江卫视黄金档。五一小长假结束后的第一天，出品方在中国传媒大学小礼堂举...,"爱情的边疆,殷桃",爱情的边疆,"['纯爱', '史诗', '登陆', '浙江', '卫视', '黄金档', '五一', '长..."
345,D000346,1分钟吸睛桥段诞生丁海寅孙艺珍“对峙”吉海妍,"人民网讯JTBC热播韩剧第10集公开了最为吸睛的1分钟桥段,这段来自于尹珍雅(孙艺珍饰)和徐...","经常请吃饭的漂亮姐姐,丁海寅,孙艺珍,吉海妍,桥段",经常请吃饭的漂亮姐姐,"['人民网', 'JTBC', '热播', '韩剧', '公开', '最为', '吸睛', ..."


### 单词计数

In [157]:
train_data['content_wordList'].head(1)

147    ['水木年华', '内地', '第一', '组合', '出自', '清华大学', '清华大学...
Name: content_wordList, dtype: object

In [158]:
def counting(item):
    word_list = literal_eval(item)
    counter = Counter(word_list).most_common()[:5]
    tag_list = [str(word[0]) for word in counter]
    return tag_list

In [159]:
Top5_content = train_data['content_wordList'].apply(counting)

In [160]:
train_data['Top5_content'] = Top5_content

In [161]:
train_data.head(10)

,ID,title,content,Tag,BookName,content_wordList,Top5_content
147,D000148,水木年华被嘲讽已过气卢庚戌回应：还好我们还没有断气！,水木年华是内地曾经的第一组合，他们都是出自清华大学。清华大学是很多莘莘学子梦寐以求的最高殿堂...,"水木年华,卢庚戌,过气",,"['水木年华', '内地', '第一', '组合', '出自', '清华大学', '清华大学...","[水木年华, 卢庚戌, 歌曲, 组合, 清华大学]"
209,D000210,著名导演陈烈院线电影即将上映,想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先来看看甄嬛...,"爱是永恒,陈列",爱是永恒,"['想必', '二刷', '三刷', '剧中', '位小主', '命运', '差异', '好...","[剧中, 婚纱照, 剧外, 娘娘, 幸福]"
211,D000212,复仇者联盟3：无限战争结局，如何影响漫威影集神盾局特工,想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先来看看甄嬛...,"复仇者联盟3,无限战争,漫威,神盾局特工",,"['想必', '二刷', '三刷', '剧中', '位小主', '命运', '差异', '好...","[剧中, 婚纱照, 剧外, 娘娘, 幸福]"
339,D000340,看片受好评殷桃回归质朴,纯爱史诗5月8日将登陆浙江卫视黄金档。五一小长假结束后的第一天，出品方在中国传媒大学小礼堂举...,"爱情的边疆,殷桃",爱情的边疆,"['纯爱', '史诗', '登陆', '浙江', '卫视', '黄金档', '五一', '长...","[殷桃, 导演, 文艺, 毛卫宁, 角色]"
345,D000346,1分钟吸睛桥段诞生丁海寅孙艺珍“对峙”吉海妍,"人民网讯JTBC热播韩剧第10集公开了最为吸睛的1分钟桥段,这段来自于尹珍雅(孙艺珍饰)和徐...","经常请吃饭的漂亮姐姐,丁海寅,孙艺珍,吉海妍,桥段",经常请吃饭的漂亮姐姐,"['人民网', 'JTBC', '热播', '韩剧', '公开', '最为', '吸睛', ...","[尹珍雅, 金美妍, 徐俊, 恋情, 吸睛]"
406,D000407,赵丽颖和杨幂假如撞档，你会不会同时追？,最近有人问：赵丽颖和杨幂假如撞档，你会不会同时追？其实和都是很受期待的电视剧，本人觉的这两部...,"赵丽颖,知否知否,杨幂,扶摇",扶摇,"['有人', '赵丽颖', '期待', '电视剧', '两部', '电视剧', '真的', ...","[赵丽颖, 该剧, 期待, 主演, 杨幂]"
430,D000431,他，绝对是我们可以爱很久的演员,第十放映室就爱瞎说大实话五一档，爱情片，票房遥遥领先。无论你是冲着导演刘若英，还是前期积攒的...,"小晓,林见清,后来的我们",,"['第十', '放映室', '瞎说', '实话', '五一', '爱情片', '票房', '...","[井柏然, 演员, 十年, 电影, 角色]"
523,D000524,“不老女神”秦海璐素颜扮可爱：今天负责萌萌哒,秦海璐素颜自拍新浪娱乐讯5月3日，秦海璐[微博]在自己微博上传自拍，配文称：“今天不工作，负...,"秦海璐,自拍,女神",,"['秦海璐', '素颜自', '新浪', '娱乐', '秦海璐', '微博', '微博', ...","[秦海璐, 微博, 萌萌, 素颜自, 新浪]"
530,D000531,靠着退票成就票房？刘若英靠情怀人设能走向成功？,晚上好，姨太的小可爱们！万万没想到，前两天还成为话题佼佼者的，在昨天却卷入了轰轰烈烈的“退票...,"刘若英,票房,退票",,"['晚上', '姨太', '小可爱', '万万', '没想到', '前两天', '话题', ...","[刘若英, 陈升, 电影, 退票, 事件]"
579,D000580,靳远套路乔梁，蔡炳坤套路逗逗,【最走心的影视人物分析，最独特的影评剧评交流】里的角色都开始调皮起来。你套路我，我考验你，玩...,"乔梁,真爱的谎言之破冰者,",真爱的谎言之破冰者,"['走心', '影视', '人物', '分析', '独特', '影评', '剧评', '交流...","[靳远, 乔梁, 怀疑, 秘密, 考验]"


In [162]:
train_data['title_wordsList'] = train_data['title'].apply(preprocess_text)

In [163]:
train_data.head()

,ID,title,content,Tag,BookName,content_wordList,Top5_content,title_wordsList
147,D000148,水木年华被嘲讽已过气卢庚戌回应：还好我们还没有断气！,水木年华是内地曾经的第一组合，他们都是出自清华大学。清华大学是很多莘莘学子梦寐以求的最高殿堂...,"水木年华,卢庚戌,过气",,"['水木年华', '内地', '第一', '组合', '出自', '清华大学', '清华大学...","[水木年华, 卢庚戌, 歌曲, 组合, 清华大学]","['水木年华', '嘲讽', '已过', '卢庚戌', '回应', '还好', '断气']"
209,D000210,著名导演陈烈院线电影即将上映,想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先来看看甄嬛...,"爱是永恒,陈列",爱是永恒,"['想必', '二刷', '三刷', '剧中', '位小主', '命运', '差异', '好...","[剧中, 婚纱照, 剧外, 娘娘, 幸福]","['著名', '导演', '陈烈', '院线', '电影', '上映']"
211,D000212,复仇者联盟3：无限战争结局，如何影响漫威影集神盾局特工,想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先来看看甄嬛...,"复仇者联盟3,无限战争,漫威,神盾局特工",,"['想必', '二刷', '三刷', '剧中', '位小主', '命运', '差异', '好...","[剧中, 婚纱照, 剧外, 娘娘, 幸福]","['复仇者', '联盟', '无限', '战争', '结局', '影响', '漫威', '影..."
339,D000340,看片受好评殷桃回归质朴,纯爱史诗5月8日将登陆浙江卫视黄金档。五一小长假结束后的第一天，出品方在中国传媒大学小礼堂举...,"爱情的边疆,殷桃",爱情的边疆,"['纯爱', '史诗', '登陆', '浙江', '卫视', '黄金档', '五一', '长...","[殷桃, 导演, 文艺, 毛卫宁, 角色]","['看片', '好评', '殷桃', '回归', '质朴']"
345,D000346,1分钟吸睛桥段诞生丁海寅孙艺珍“对峙”吉海妍,"人民网讯JTBC热播韩剧第10集公开了最为吸睛的1分钟桥段,这段来自于尹珍雅(孙艺珍饰)和徐...","经常请吃饭的漂亮姐姐,丁海寅,孙艺珍,吉海妍,桥段",经常请吃饭的漂亮姐姐,"['人民网', 'JTBC', '热播', '韩剧', '公开', '最为', '吸睛', ...","[尹珍雅, 金美妍, 徐俊, 恋情, 吸睛]","['分钟', '吸睛', '桥段', '诞生', '丁海寅', '孙艺珍', '对峙', '..."


In [167]:
Top5_content_list = train_data['Top5_content'].values.tolist()

In [168]:
title_wordsListofList = train_data['title_wordsList'].values.tolist()

In [170]:
title_wordsListofList = [literal_eval(word_list) for word_list in title_wordsListofList]

### intersection

In [173]:
intersectio_listofList = []
for (contentWordlist, titleWordlist) in zip(Top5_content_list,title_wordsListofList ):
    intersection_list = [word for word in contentWordlist if word in titleWordlist]
    intersectio_listofList.append(intersection_list)

In [176]:
train_data['intersection'] = intersectio_listofList

In [177]:
train_data.head()

,ID,title,content,Tag,BookName,content_wordList,Top5_content,title_wordsList,intersection
147,D000148,水木年华被嘲讽已过气卢庚戌回应：还好我们还没有断气！,水木年华是内地曾经的第一组合，他们都是出自清华大学。清华大学是很多莘莘学子梦寐以求的最高殿堂...,"水木年华,卢庚戌,过气",,"['水木年华', '内地', '第一', '组合', '出自', '清华大学', '清华大学...","[水木年华, 卢庚戌, 歌曲, 组合, 清华大学]","['水木年华', '嘲讽', '已过', '卢庚戌', '回应', '还好', '断气']","[水木年华, 卢庚戌]"
209,D000210,著名导演陈烈院线电影即将上映,想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先来看看甄嬛...,"爱是永恒,陈列",爱是永恒,"['想必', '二刷', '三刷', '剧中', '位小主', '命运', '差异', '好...","[剧中, 婚纱照, 剧外, 娘娘, 幸福]","['著名', '导演', '陈烈', '院线', '电影', '上映']",[]
211,D000212,复仇者联盟3：无限战争结局，如何影响漫威影集神盾局特工,想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先来看看甄嬛...,"复仇者联盟3,无限战争,漫威,神盾局特工",,"['想必', '二刷', '三刷', '剧中', '位小主', '命运', '差异', '好...","[剧中, 婚纱照, 剧外, 娘娘, 幸福]","['复仇者', '联盟', '无限', '战争', '结局', '影响', '漫威', '影...",[]
339,D000340,看片受好评殷桃回归质朴,纯爱史诗5月8日将登陆浙江卫视黄金档。五一小长假结束后的第一天，出品方在中国传媒大学小礼堂举...,"爱情的边疆,殷桃",爱情的边疆,"['纯爱', '史诗', '登陆', '浙江', '卫视', '黄金档', '五一', '长...","[殷桃, 导演, 文艺, 毛卫宁, 角色]","['看片', '好评', '殷桃', '回归', '质朴']",[殷桃]
345,D000346,1分钟吸睛桥段诞生丁海寅孙艺珍“对峙”吉海妍,"人民网讯JTBC热播韩剧第10集公开了最为吸睛的1分钟桥段,这段来自于尹珍雅(孙艺珍饰)和徐...","经常请吃饭的漂亮姐姐,丁海寅,孙艺珍,吉海妍,桥段",经常请吃饭的漂亮姐姐,"['人民网', 'JTBC', '热播', '韩剧', '公开', '最为', '吸睛', ...","[尹珍雅, 金美妍, 徐俊, 恋情, 吸睛]","['分钟', '吸睛', '桥段', '诞生', '丁海寅', '孙艺珍', '对峙', '...",[吸睛]


### 2) 人名提取

In [223]:
titles = train_data['title_wordsList'].values.tolist()

In [226]:
literal_eval(titles[0])

['水木年华', '嘲讽', '已过', '卢庚戌', '回应', '还好', '断气']

In [227]:
titles_1 = [literal_eval(title) for title in titles]

In [241]:
name_listofList = []
for title_list in titles_1:
    name_list = []
    for name in title_list:
        pair= pseg.cut(name)
        pair = list(pair)[0]
        if pair.flag == 'nr':
            name_list.append(pair.word)
    name_listofList.append(name_list)

In [1]:
train_data['person_name'] = name_listofList

NameError: name 'name_listofList' is not defined

In [244]:
train_data.head()

,ID,title,content,Tag,BookName,content_wordList,Top5_content,title_wordsList,intersection,person_name
147,D000148,水木年华被嘲讽已过气卢庚戌回应：还好我们还没有断气！,水木年华是内地曾经的第一组合，他们都是出自清华大学。清华大学是很多莘莘学子梦寐以求的最高殿堂...,"水木年华,卢庚戌,过气",,"['水木年华', '内地', '第一', '组合', '出自', '清华大学', '清华大学...","[水木年华, 卢庚戌, 歌曲, 组合, 清华大学]","['水木年华', '嘲讽', '已过', '卢庚戌', '回应', '还好', '断气']","[水木年华, 卢庚戌]",[]
209,D000210,著名导演陈烈院线电影即将上映,想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先来看看甄嬛...,"爱是永恒,陈列",爱是永恒,"['想必', '二刷', '三刷', '剧中', '位小主', '命运', '差异', '好...","[剧中, 婚纱照, 剧外, 娘娘, 幸福]","['著名', '导演', '陈烈', '院线', '电影', '上映']",[],[陈烈]
211,D000212,复仇者联盟3：无限战争结局，如何影响漫威影集神盾局特工,想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先来看看甄嬛...,"复仇者联盟3,无限战争,漫威,神盾局特工",,"['想必', '二刷', '三刷', '剧中', '位小主', '命运', '差异', '好...","[剧中, 婚纱照, 剧外, 娘娘, 幸福]","['复仇者', '联盟', '无限', '战争', '结局', '影响', '漫威', '影...",[],[]
339,D000340,看片受好评殷桃回归质朴,纯爱史诗5月8日将登陆浙江卫视黄金档。五一小长假结束后的第一天，出品方在中国传媒大学小礼堂举...,"爱情的边疆,殷桃",爱情的边疆,"['纯爱', '史诗', '登陆', '浙江', '卫视', '黄金档', '五一', '长...","[殷桃, 导演, 文艺, 毛卫宁, 角色]","['看片', '好评', '殷桃', '回归', '质朴']",[殷桃],[殷桃]
345,D000346,1分钟吸睛桥段诞生丁海寅孙艺珍“对峙”吉海妍,"人民网讯JTBC热播韩剧第10集公开了最为吸睛的1分钟桥段,这段来自于尹珍雅(孙艺珍饰)和徐...","经常请吃饭的漂亮姐姐,丁海寅,孙艺珍,吉海妍,桥段",经常请吃饭的漂亮姐姐,"['人民网', 'JTBC', '热播', '韩剧', '公开', '最为', '吸睛', ...","[尹珍雅, 金美妍, 徐俊, 恋情, 吸睛]","['分钟', '吸睛', '桥段', '诞生', '丁海寅', '孙艺珍', '对峙', '...",[吸睛],"[丁海寅, 孙艺珍, 吉海妍]"


In [269]:
def Pred_Tag(BookName,intersection,person_name):
    list_pred = []
    list_pred.append(BookName)
    list_pred.extend(intersection)
    list_pred.extend(person_name)
    list_pred = np.unique(list_pred)
    string = ''
    for word in list_pred:
        string = string+word+','

    string = string[:-1]
    try:
        if string[0] ==',':
            string = string[1:]
    except IndexError:
        pass
    
    return string

In [270]:
pred_tag = train_data[['BookName','intersection','person_name']].apply(lambda x: Pred_Tag(x[0],x[1],x[2]),axis = 1)

In [271]:
pred_tag.head()

147                     卢庚戌,水木年华
209                      爱是永恒,陈烈
211                             
339                     殷桃,爱情的边疆
345    丁海寅,吉海妍,吸睛,孙艺珍,经常请吃饭的漂亮姐姐
dtype: object

In [272]:
train_data['pre_tag'] = pred_tag

In [273]:
train_data.head(10)

,ID,title,content,Tag,BookName,content_wordList,Top5_content,title_wordsList,intersection,person_name,pre_tag
147,D000148,水木年华被嘲讽已过气卢庚戌回应：还好我们还没有断气！,水木年华是内地曾经的第一组合，他们都是出自清华大学。清华大学是很多莘莘学子梦寐以求的最高殿堂...,"水木年华,卢庚戌,过气",,"['水木年华', '内地', '第一', '组合', '出自', '清华大学', '清华大学...","[水木年华, 卢庚戌, 歌曲, 组合, 清华大学]","['水木年华', '嘲讽', '已过', '卢庚戌', '回应', '还好', '断气']","[水木年华, 卢庚戌]",[],"卢庚戌,水木年华"
209,D000210,著名导演陈烈院线电影即将上映,想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先来看看甄嬛...,"爱是永恒,陈列",爱是永恒,"['想必', '二刷', '三刷', '剧中', '位小主', '命运', '差异', '好...","[剧中, 婚纱照, 剧外, 娘娘, 幸福]","['著名', '导演', '陈烈', '院线', '电影', '上映']",[],[陈烈],"爱是永恒,陈烈"
211,D000212,复仇者联盟3：无限战争结局，如何影响漫威影集神盾局特工,想必很多人已经二刷三刷，剧中5位小主的命运差异好大，剧外的她们，婚纱照都相差好大先来看看甄嬛...,"复仇者联盟3,无限战争,漫威,神盾局特工",,"['想必', '二刷', '三刷', '剧中', '位小主', '命运', '差异', '好...","[剧中, 婚纱照, 剧外, 娘娘, 幸福]","['复仇者', '联盟', '无限', '战争', '结局', '影响', '漫威', '影...",[],[],
339,D000340,看片受好评殷桃回归质朴,纯爱史诗5月8日将登陆浙江卫视黄金档。五一小长假结束后的第一天，出品方在中国传媒大学小礼堂举...,"爱情的边疆,殷桃",爱情的边疆,"['纯爱', '史诗', '登陆', '浙江', '卫视', '黄金档', '五一', '长...","[殷桃, 导演, 文艺, 毛卫宁, 角色]","['看片', '好评', '殷桃', '回归', '质朴']",[殷桃],[殷桃],"殷桃,爱情的边疆"
345,D000346,1分钟吸睛桥段诞生丁海寅孙艺珍“对峙”吉海妍,"人民网讯JTBC热播韩剧第10集公开了最为吸睛的1分钟桥段,这段来自于尹珍雅(孙艺珍饰)和徐...","经常请吃饭的漂亮姐姐,丁海寅,孙艺珍,吉海妍,桥段",经常请吃饭的漂亮姐姐,"['人民网', 'JTBC', '热播', '韩剧', '公开', '最为', '吸睛', ...","[尹珍雅, 金美妍, 徐俊, 恋情, 吸睛]","['分钟', '吸睛', '桥段', '诞生', '丁海寅', '孙艺珍', '对峙', '...",[吸睛],"[丁海寅, 孙艺珍, 吉海妍]","丁海寅,吉海妍,吸睛,孙艺珍,经常请吃饭的漂亮姐姐"
406,D000407,赵丽颖和杨幂假如撞档，你会不会同时追？,最近有人问：赵丽颖和杨幂假如撞档，你会不会同时追？其实和都是很受期待的电视剧，本人觉的这两部...,"赵丽颖,知否知否,杨幂,扶摇",扶摇,"['有人', '赵丽颖', '期待', '电视剧', '两部', '电视剧', '真的', ...","[赵丽颖, 该剧, 期待, 主演, 杨幂]",['赵丽颖'],[赵丽颖],[赵丽颖],"扶摇,赵丽颖"
430,D000431,他，绝对是我们可以爱很久的演员,第十放映室就爱瞎说大实话五一档，爱情片，票房遥遥领先。无论你是冲着导演刘若英，还是前期积攒的...,"小晓,林见清,后来的我们",,"['第十', '放映室', '瞎说', '实话', '五一', '爱情片', '票房', '...","[井柏然, 演员, 十年, 电影, 角色]",['演员'],[演员],[],演员
523,D000524,“不老女神”秦海璐素颜扮可爱：今天负责萌萌哒,秦海璐素颜自拍新浪娱乐讯5月3日，秦海璐[微博]在自己微博上传自拍，配文称：“今天不工作，负...,"秦海璐,自拍,女神",,"['秦海璐', '素颜自', '新浪', '娱乐', '秦海璐', '微博', '微博', ...","[秦海璐, 微博, 萌萌, 素颜自, 新浪]","['不老', '女神', '秦海璐', '素颜扮', '可爱', '负责', '萌萌']","[秦海璐, 萌萌]",[秦海璐],"秦海璐,萌萌"
530,D000531,靠着退票成就票房？刘若英靠情怀人设能走向成功？,晚上好，姨太的小可爱们！万万没想到，前两天还成为话题佼佼者的，在昨天却卷入了轰轰烈烈的“退票...,"刘若英,票房,退票",,"['晚上', '姨太', '小可爱', '万万', '没想到', '前两天', '话题', ...","[刘若英, 陈升, 电影, 退票, 事件]","['靠着', '退票', '成就', '票房', '刘若英', '情怀', '人设', '走...","[刘若英, 退票]",[刘若英],"刘若英,退票"
579,D000580,靳远套路乔梁，蔡炳坤套路逗逗,【最走心的影视人物分析，最独特的影评剧评交流】里的角色都开始调皮起来。你套路我，我考验你，玩...,"乔梁,真爱的谎言之破冰者,",真爱的谎言之破冰者,"['走心', '影视', '人物', '分析', '独特', '影评', '剧评', '交流...","[靳远, 乔梁, 怀疑, 秘密, 考验]","['靳远', '套路', '乔梁', '蔡炳坤', '套路', '逗逗']","[靳远, 乔梁]","[靳远, 乔梁, 蔡炳]","乔梁,真爱的谎言之破冰者,蔡炳,靳远"


### Prediction

In [274]:
pred_data.head()

,ID,title,content,Tag
0,D000001,林志颖老婆深夜敷面膜，睫毛太长好吓人,早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...,NaN
1,D000002,小s夸杨幂身材好，杨幂回复太精彩了！,翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...,NaN
2,D000003,年轻时的她风华绝代，现却无人送祝福,上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...,NaN
3,D000004,林心如屡曝霍建华私生活被怼蹭老公人气,霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...,NaN
4,D000005,曾是TVB颜值担当，近照曝光发现真老了,不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...,NaN


### 1) 获取书名

In [275]:
pred_data.to_csv('pre_data.csv',index = False)